# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




In [9]:
%%capture
!pip install dlt[duckdb] # Install dlt with all the necessary DuckDB dependencies

# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [4]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

total=0
for sqrt_value in generator:
    total+=sqrt_value
    print(sqrt_value)

print(total)


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
8.382332347441762


In [ ]:
13**0.5

3.605551275463989

### Solution 1

#### Question 1: What is the sum of the outputs of the generator for limit = 5?


In [7]:
limit = 5
generator1 = square_root_generator(limit)
result = sum(next(generator1) for _ in range(limit))
print(f"The sum of the outputs for limit={limit} is: {result}")

The sum of the outputs for limit=5 is: 8.382332347441762


#### Question 2: What is the 13th number yielded by the generator?

In [6]:
n = 13
generator_2 = square_root_generator(n)
for _ in range(n - 1):
    next(generator_2)

number = next(generator_2)
print(number)

3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [10]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


### Solution 2:

In [13]:
import dlt
import duckdb

#### Question 3: Append the 2 generators. After correctly appending the data, calculate the sum of all ages of people.

In [22]:
# Create a DLT pipeline for the first generator `people_1`
# The pipeline is set to load data into a DuckDB database with the dataset named 'people'
load_people_1 = dlt.pipeline(destination='duckdb', dataset_name='people')

# Run the pipeline for the first generator, creating or replacing the table 'people'
load_people_1.run(people_1(),
                  table_name="people",
                  write_disposition="replace")

LoadInfo(pipeline=<dlt.pipeline.pipeline.Pipeline object at 0x7c17b124fbb0>, metrics={'1707346114.9078457': [{'started_at': DateTime(2024, 2, 7, 22, 48, 35, 296910, tzinfo=Timezone('UTC')), 'finished_at': DateTime(2024, 2, 7, 22, 48, 35, 565245, tzinfo=Timezone('UTC'))}]}, destination_type='dlt.destinations.duckdb', destination_displayable_credentials='duckdb:////content/dlt_colab_kernel_launcher.duckdb', destination_name='duckdb', environment=None, staging_type=None, staging_name=None, staging_displayable_credentials=None, destination_fingerprint='', dataset_name='people', loads_ids=['1707346114.9078457'], load_packages=[LoadPackageInfo(load_id='1707346114.9078457', package_path='/var/dlt/pipelines/dlt_colab_kernel_launcher/load/loaded/1707346114.9078457', state='loaded', schema=Schema dlt_colab_kernel_launcher at 136441092879936, schema_update={}, completed_at=DateTime(2024, 2, 7, 22, 48, 35, 553952, tzinfo=Timezone('UTC')), jobs={'completed_jobs': [LoadJobInfo(state='completed_jobs'

In [23]:
# Connect to the DuckDB database
conn = duckdb.connect(f"{load_people_1.pipeline_name}.duckdb")
# Extracting the data from the 'people' table
data = conn.sql(f"SELECT * FROM {load_people_1.dataset_name}.people").df()
display(data)

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707346114.9078457,2YSPX+9zWjloXA,None
1,2,Person_2,27,City_A,1707346114.9078457,qFk8S4ykqeWjPQ,None
2,3,Person_3,28,City_A,1707346114.9078457,osItultxYDC7zg,None
3,4,Person_4,29,City_A,1707346114.9078457,rPlH1O+e7HDK2w,None
4,5,Person_5,30,City_A,1707346114.9078457,47Vx9/md4GRBsw,None


Append the `people_2`generator

In [24]:
# Create a  DLT pipeline for people_2, to the same dataset
load_people_2 = dlt.pipeline(destination='duckdb', dataset_name='people')

# Run the second pipeline, appending data the data from people_2
load_people_2.run(people_2(),
                  table_name="people",
                  write_disposition="append")


LoadInfo(pipeline=<dlt.pipeline.pipeline.Pipeline object at 0x7c17b124df00>, metrics={'1707346124.6896133': [{'started_at': DateTime(2024, 2, 7, 22, 48, 45, 128875, tzinfo=Timezone('UTC')), 'finished_at': DateTime(2024, 2, 7, 22, 48, 45, 353716, tzinfo=Timezone('UTC'))}]}, destination_type='dlt.destinations.duckdb', destination_displayable_credentials='duckdb:////content/dlt_colab_kernel_launcher.duckdb', destination_name='duckdb', environment=None, staging_type=None, staging_name=None, staging_displayable_credentials=None, destination_fingerprint='', dataset_name='people', loads_ids=['1707346124.6896133'], load_packages=[LoadPackageInfo(load_id='1707346124.6896133', package_path='/var/dlt/pipelines/dlt_colab_kernel_launcher/load/loaded/1707346124.6896133', state='loaded', schema=Schema dlt_colab_kernel_launcher at 136441093031680, schema_update={}, completed_at=DateTime(2024, 2, 7, 22, 48, 45, 333609, tzinfo=Timezone('UTC')), jobs={'completed_jobs': [LoadJobInfo(state='completed_jobs'

In [25]:
# Extracting the data from the 'people' table
data = conn.sql(f"SELECT * FROM {load_people_2.dataset_name}.people").df()
display(data)
# Calculate the sum of ages from the 'people' table
sum_ages = conn.execute(f"SELECT SUM(age) FROM {load_people_2.dataset_name}.people").fetchone()[0]
print(f"Sum of ages from generators: {sum_ages}")

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707346114.9078457,2YSPX+9zWjloXA,None
1,2,Person_2,27,City_A,1707346114.9078457,qFk8S4ykqeWjPQ,None
2,3,Person_3,28,City_A,1707346114.9078457,osItultxYDC7zg,None
3,4,Person_4,29,City_A,1707346114.9078457,rPlH1O+e7HDK2w,None
4,5,Person_5,30,City_A,1707346114.9078457,47Vx9/md4GRBsw,None
5,3,Person_3,33,City_B,1707346124.6896133,36kriQA64eEp6w,Job_3
6,4,Person_4,34,City_B,1707346124.6896133,BrDnx6T/+1QYow,Job_4
7,5,Person_5,35,City_B,1707346124.6896133,Ch3PLeU29LxslQ,Job_5
8,6,Person_6,36,City_B,1707346124.6896133,2gigjfA0f5gu6Q,Job_6
9,7,Person_7,37,City_B,1707346124.6896133,mPZS13Kff18oAw,Job_7


Sum of ages from generators: 353


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


### Solution 3:

#### Question 4: Merge the 2 generators using the ID column. Calculate the sum of ages of all the people loaded as described above.

In [26]:
# Create the pipeline.
merge_pipeline = dlt.pipeline(destination='duckdb', dataset_name='merge_people')

# Load data from the first generator `people_1` into 'people_merge' table.
# This operation will replace any existing data in the table.
# A primary key 'ID' is specified for potential future merge operations.
merge_pipeline.run(people_1(),
                   table_name="merge_people",
                   write_disposition="replace",
                   primary_key="ID")

# Load data from the second generator `people_2` into the same 'people_merge' table.
# This operation will merge the new data with existing data based on the primary key 'ID'.
merge_pipeline.run(people_2(),
                  table_name="merge_people",
                  write_disposition="merge",
                  primary_key="ID")

LoadInfo(pipeline=<dlt.pipeline.pipeline.Pipeline object at 0x7c17b006e4d0>, metrics={'1707346488.1945899': [{'started_at': DateTime(2024, 2, 7, 22, 54, 48, 588869, tzinfo=Timezone('UTC')), 'finished_at': DateTime(2024, 2, 7, 22, 54, 49, 11253, tzinfo=Timezone('UTC'))}]}, destination_type='dlt.destinations.duckdb', destination_displayable_credentials='duckdb:////content/dlt_colab_kernel_launcher.duckdb', destination_name='duckdb', environment=None, staging_type=None, staging_name=None, staging_displayable_credentials=None, destination_fingerprint='', dataset_name='merge_people', loads_ids=['1707346488.1945899'], load_packages=[LoadPackageInfo(load_id='1707346488.1945899', package_path='/var/dlt/pipelines/dlt_colab_kernel_launcher/load/loaded/1707346488.1945899', state='loaded', schema=Schema dlt_colab_kernel_launcher at 136441092878016, schema_update={'merge_people': {'name': 'merge_people', 'columns': {'occupation': {'name': 'occupation', 'data_type': 'text', 'nullable': True}}, 'writ

Show the results

In [27]:
# Establish a connection to the DuckDB database created by the pipeline.
conn = duckdb.connect(f"{merge_pipeline.pipeline_name}.duckdb")

# Display the merged data from the 'people_merged' table.
print("Data from the merge_people table:")
data = conn.sql(f"SELECT * FROM {merge_pipeline.dataset_name}.merge_people").df()
display(data)

# Calculate and display the sum of ages from the merged data in 'people_merged' table.
sum_ages = conn.execute(f"SELECT SUM(age) FROM {merge_pipeline.dataset_name}.merge_people").fetchone()[0]
print(f"Sum of ages of merge_people is: {sum_ages}")


Data from the merge_people table:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707346487.4630766,UaYXixz8WONsPA,None
1,2,Person_2,27,City_A,1707346487.4630766,rd+cq5HrZghVVQ,None
2,5,Person_5,35,City_B,1707346488.1945899,q399TYv78ngUpQ,Job_5
3,8,Person_8,38,City_B,1707346488.1945899,mwLTm8ZfeA42zg,Job_8
4,7,Person_7,37,City_B,1707346488.1945899,IgC7DqT6LEgERw,Job_7
5,4,Person_4,34,City_B,1707346488.1945899,/UtPbGQf0yLMeg,Job_4
6,3,Person_3,33,City_B,1707346488.1945899,xzaDV/VX2YVhMw,Job_3
7,6,Person_6,36,City_B,1707346488.1945899,5cjsJVkgHQn8ag,Job_6


Sum of ages of merge_people is: 266


Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX